# Compute Magnitudes Attenuation for O2 x airmass  from Flat SED in LSST filters

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2024/12/05
- last update : 2024/12/05 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
from scipy import interpolate

In [ ]:
machine_name = os.uname().nodename
dm_version = "w_2024_38"
path_rubinsimphot = f"repos/repos_{dm_version}/rubinsimphot/src"
#path_rubinsimphot = "repos/repos_w_2024_17/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py310")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

In [ ]:
def Get_SED_Pickles():
    seddir = os.path.join(fdir, 'pysynphot', 'pickles')
    seddir_uvi = os.path.join(seddir,"dat_uvi")
    seddir_uvk = os.path.join(seddir,"dat_uvk")
    all_pickles_uvi = sorted(os.listdir(seddir_uvi))
    all_pickles_uvk = sorted(os.listdir(seddir_uvk))
    file_ref = os.path.join(seddir_uvk, "pickles_uk.fits")
    hdul = fits.open(file_ref)
    df_pickle = pd.DataFrame(hdul[1].data)
    NSED = len(df_pickle)

    for index in np.arange(NSED):
        filename = df_pickle.loc[index,"FILENAME"].strip()+".fits"
        fullfilename = os.path.join(seddir_uvk,filename) 
        hdul = fits.open(fullfilename)
        dff = pd.DataFrame(hdul[1].data)

In [ ]:
# reference flux in Jy
F0 = ((0.*u.ABmag).to(u.Jy)).value
F0

## Imports dedicated to this work

- import the atmospheric transparency emulator (instead of using libradtran code).
- import rubin sim
- import libPhotometricCorrections : encapsulate uninteresting calculation details

### libradtran Emulator

In [ ]:
from importlib.metadata import version
the_ver = version('getObsAtmo')
print(f"Version of getObsAtmo : {the_ver}")

In [ ]:
from getObsAtmo import ObsAtmo
emul = ObsAtmo("LSST")

In [ ]:
WL = emul.GetWL()

#### Library to fit atmosphere

In [ ]:
import sys
sys.path.append('../lib')
#import libAtmosphericFit

#### Library that encapsulate calculations for Photometric correction

In [ ]:
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections import *

In [ ]:
def set_photometric_parameters(exptime, nexp, readnoise=None):
    # readnoise = None will use the default (8.8 e/pixel). Readnoise should be in electrons/pixel.
    photParams = PhotometricParameters(exptime=exptime, nexp=nexp, readnoise=readnoise)
    return photParams

In [ ]:
def scale_sed(ref_mag, ref_filter, sed):
    fluxNorm = sed.calc_flux_norm(ref_mag, lsst_std[ref_filter])
    sed.multiply_flux_norm(fluxNorm)
    return sed

In [ ]:
# set default photometric parameters to compute ADU
photoparams = set_photometric_parameters(30, 1 , readnoise=None)

#### library rubin_sim defining LSST parameters, namely for photometric calculations

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data import get_data_dir

## Configuration

In [ ]:
am0 = 1.20    # airmass
pwv0 = 0.0  # Precipitable water vapor vertical column depth in mm
oz0 = 0.  # Ozone vertical column depth in Dobson Unit (DU)
ncomp=1     # Number of aerosol components
tau0= 0.0 # Vertical Aerosol depth (VAOD) 
beta0 = 1.2 # Aerosol Angstrom exponent

### Initialisation of Atmospheric corrections

In [ ]:
pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)

### Check standard atmosphere

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
axs.plot(pc.WL,pc.atm_std,'k-')
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Standard atmosphere transmission")

### Check LSST instrument throughput

Photometric Correction package should find the instrumental passband of LSST

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Instrument throughput (auxtel)")

### Check LSST standard Filter throughputs

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Total filter throughput (auxtel)")

## Range of airmasses 

In [ ]:
#all_airmasses = np.linspace(1.,2.5,26)
all_airmasses = np.arange(1,2.5,0.01)
pwv = pwv0
oz = oz0
tau= tau0
beta = beta0
NAM = len(all_airmasses)

In [ ]:
all_airmasses

## Generate multi-observations

- for all airmasses

In [ ]:
all_pc = []
all_pc0 = []
all_am = []
for idx_am,am in enumerate(all_airmasses): 
    # create a new PhotometricCorrections by defining its standard parameters
    the_pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)
    the_pc0 = PhotometricCorrections(am,pwv0,oz0,tau0,beta0) # standard atmosphere but at airmass am

    # calculate the transmissions at a given airmass am for all PWV values
    #the_pc.CalculateMultiObs(am,pwv,oz,tau,beta)
    #the_pc0.CalculateMultiObs(am,pwv,oz,tau,beta)
    
    all_pc.append(the_pc)
    all_pc0.append(the_pc0)

    # compute the products airmass 
    all_am.append(am) 

In [ ]:
# Figure 1
jet = plt.get_cmap('jet')
NAM = len(all_airmasses)
cNorm = colors.Normalize(vmin=0, vmax=NAM)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NAM), alpha=1)

difference_airmass = np.absolute(all_airmasses-am0)
idx_am = difference_airmass.argmin()
the_am = all_airmasses[idx_am]
the_pc = all_pc[idx_am]



fig = plt.figure(figsize=(7,5),layout='constrained')
ax = fig.add_subplot(1,1,1)
for idx_am,am in enumerate(all_airmasses): 
    the_pc0 = all_pc0[idx_am]    
    ax.plot(the_pc0.WL,the_pc0.atm_std,color=all_colors[idx_am],lw=0.5)

ax.plot(the_pc.WL,the_pc.atm_std,color="k",lw=0.5,label="standard atmosphere")

ax.legend()
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("atmospheric transmission")
ax.set_title("standard atm transmission at diff airmass (1-2.5)")

ax2 = ax.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(the_pc0.bandpass_total_std[f].wavelen,the_pc0.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])
    
plt.show()

### PWV variation :  Observed filter and normalized response

In [ ]:

# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = colors.Normalize(vmin=0, vmax=NOBS)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


fig = plt.figure(figsize=(12,5),layout='constrained')

#find the index of the closest airmas for standard atmosphere
difference_airmass = np.absolute(all_airmasses-am0)
idx_am = difference_airmass.argmin()
the_am = all_airmasses[idx_am]
the_pc = all_pc[idx_am]


# Figure 1
axs=fig.add_subplot(1,2,1)

  
atm = the_pc.atm_std
    
label = f"am = {the_am:.3f}" 
axs.plot(the_pc.WL,atm,label=label,lw=0.5)
   
axs.plot(the_pc.WL,the_pc.atm_std,color="k",lw=2,label="standard atmosphere")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  
axs.set_xlabel("$\lambda$ (nm)")
axs.set_ylabel("atmospheric transmission")
axs.set_title("standard and observed transmission")

ax2 = axs.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(the_pc.bandpass_total_std[f].wavelen,the_pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])

# Figure 2
axs=fig.add_subplot(1,2,2)

all_linestyles = ['-','--','-.',':','-','--','-.',':','-','--','-.',':','-','--','-.',':']

# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = colors.Normalize(vmin=0, vmax=NOBS)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)



    
for ifilt,f in enumerate(filter_tagnames):
        
    the_x=the_pc.WL
    the_y=the_pc.phiArray_std[ifilt,:]
       
        
    if ifilt==1:
        axs.plot(the_x,the_y,linestyle="-",label=label )
    else:
        axs.plot(the_x,the_y,linestyle="-")

axs.set_xlabel("$\lambda$ (nm)")
axs.set_ylabel("$\\phi_b(\lambda)$")
axs.set_title("Normalized observed transmission")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  


#plt.tight_layout()
plt.show()



In [ ]:
#NOBS = len(all_pwv)

# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = colors.Normalize(vmin=0, vmax=NOBS)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


fig = plt.figure(figsize=(6,5))

# Figure 1
axs=fig.add_subplot(1,1,1)

atm_bands = the_pc.bandpass_total_std    

for f in filter_tagnames: 
    axs.plot(atm_bands[f].wavelen,atm_bands[f].sb,color=all_colors[index],label=label,lw=0.5)

axs.plot(the_pc.WL,the_pc.atm_std,color="k",lw=2,label="standard atmosphere")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  
axs.set_xlabel("$\lambda$ (nm)")
axs.set_ylabel("atmospheric transmission")

axs.set_title(f"standard and observed transmission for airmass {the_am:.2f}")

ax2 = axs.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(the_pc.bandpass_total_std[f].wavelen,the_pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])


plt.tight_layout()
plt.show()


## SED

In [ ]:
# Find the throughputs directory 
#fdir = os.getenv('RUBIN_SIM_DATA_DIR')
fdir = get_data_dir()
if fdir is None:  #environment variable not set
    fdir = os.path.join(os.getenv('HOME'), 'rubin_sim_data')

In [ ]:
the_sed_flat = Sed()
the_sed_flat.set_flat_sed()
the_sed_flat.name = 'flat'
zmag = 20.0
flux_norm = the_sed_flat.calc_flux_norm(zmag, pc.bandpass_total_std['z'])
the_sed_flat.multiply_flux_norm(flux_norm)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
ax.plot(the_sed_flat .wavelen,-2.5*np.log10(the_sed_flat.fnu/F0),"b-",label=the_sed_flat.name)

ax.legend()
#ax.set_ylim(1e-17,1e-14)
#ax.set_xlim(300.,2000.)
ax.set_title("Flat SED $F_\\nu$")
ax.set_ylabel(" Magnitude = $-2.5 \log_{10}(F_\\nu/F_0)$")
ax.set_xlabel("$\\lambda \, (nm)$")
ax.yaxis.set_inverted(True)


ax3 = ax.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax3.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax3.set_yticks([])

In [ ]:
the_sed = the_sed_flat

In [ ]:
fig,(ax,ax2) = plt.subplots(1,2,figsize=(16,6))
ax.plot(the_sed .wavelen,the_sed .flambda,"b-",label=the_sed.name)
ax.legend()
ax.set_ylim(1e-17,1e-15)
ax.set_xlim(300.,1200.)
ax.set_title("Flast SED $F_\lambda$")
ax.set_ylabel("$F_\lambda$")
ax.set_xlabel("$\lambda \, (nm)$")


ax2.plot(the_sed .wavelen,the_sed.fnu,"b-",label=the_sed.name)

ax2.set_yscale("log")
ax2.legend()
ax2.set_ylim(1e-5,1e-4)
ax2.set_xlim(300.,1200.)
ax2.set_title("Flat $F_\\nu$")
ax2.set_ylabel("$F_\\nu$")
ax2.set_xlabel("$\lambda \, (nm)$")

ax3 = ax.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax3.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax3.set_yticks([])
    
ax4 = ax2.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax4.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax4.set_yticks([])

## Calculate magnitudes AB and Observed Magnitudes for standard and non-standard Magnitudes 

- by construction the AB magnitudes are invariant as the true transmission is considered
- the observed magnitudes provide the resolution as the true transmission-observed transmision is not the expected transmission

In [ ]:
mag_std = {}
adu_std = {}
atm_bands = the_pc.bandpass_total_std
for index,f in enumerate(filter_tagnames) :
    mag_std[f] = the_sed.calc_mag(atm_bands[f])
    adu_std[f] = -2.5*np.log10(the_sed.calc_adu(atm_bands[f],photoparams))

In [ ]:
mag_std

In [ ]:
adu_std

#### Generate magntudes for non standard  atmosphere for all airmass and all pwv

In [ ]:
all_df = []

# loop on airmass
for idx_am,am in enumerate(all_airmasses): 
    # create a new PhotometricCorrections by defining its standard parameters
    
    the_pc = all_pc[idx_am]  # not standard atmosphere
    the_pc0 = all_pc0[idx_am] # standard atmosphere but at airmass am
    

    # the standard atmosphere at this airmass
    atm_bands_std = the_pc0.bandpass_total_std

    df = pd.DataFrame(columns = ["am","aduu_std","adug_std","adur_std","adui_std","aduz_std","aduy_std"])

    #calculate the ADU for the standard atmosphere at this airmass am
    adu_std = {}
    for index,f in enumerate(filter_tagnames):
        adu_std[f] = -2.5*np.log10(the_sed.calc_adu(atm_bands_std[f],photoparams)) # this is really the calculation through the physical filter


        
    df.loc[0] = [am, adu_std["u"],adu_std["g"],adu_std["r"],adu_std["i"],adu_std["z"],adu_std["y"]] 

    all_df.append(df)

In [ ]:
df_merge = pd.concat(all_df)
df_merge.reset_index(inplace=True)

In [ ]:
df_merge

### Compute difference in mmag between the value and the standard atmosphere value at airmass am0

In [ ]:
#for index,f in enumerate(filter_tagnames) :
#    label_in = f'adu{f}'
#    label_in_std = f'adu{f}_std'
#    label_in_0 = f'adu{f}_0'
#    label_out1 =f'd_adu{f}_mag'
#    label_out2 =f'd_adu_0{f}_mag'
#    label_out3 =f'd_adu{f}_mmag'
#    label_out4 =f'd_adu_0{f}_mmag'
    #df_merge[label_out] = (df_merge[label_in]- adu_std[f])*1000. 
#    df_merge[label_out1] = (df_merge[label_in]- df_merge[label_in_std]) # keep in mag
#    df_merge[label_out2] = (df_merge[label_in]- df_merge[label_in_0])  # keep in mag
#    df_merge[label_out3] = (df_merge[label_in]- df_merge[label_in_std])*1000.  # converted in mmag
#    df_merge[label_out4] = (df_merge[label_in]- df_merge[label_in_0])*1000.  # converted in mmag

In [ ]:
df_merge

### Drop absolute mags and keep mag difference

In [ ]:
#df = df.drop(labels=["aduu","adug","adur","adui","aduz","aduy"],axis=1)

In [ ]:
#fig,ax = plt.subplots(1,1,figsize=(10,6),layout="constrained")
#df_merge.plot.scatter(x="amxpwv", y="magy",ax=ax,marker=".",grid=True,c='grey',label="Y") 
#df_merge.plot.scatter(x="amxpwv", y="magz",ax=ax,marker=".",grid=True,c='k',label="Z") 
#df_merge.plot.scatter(x="amxpwv", y="magi",ax=ax,marker=".",grid=True,c='orange',label="I") 

### Relative Extinction in -2.5 log(ADU) relative to standard atmosphere at observed airmass vs (airmass x PWV)

- Relative Extinction means subtract the standard atmosphere ADU at observed airmass

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6),layout="constrained")

df_merge.plot.scatter(x="amxpwv", y="d_aduu_mmag",ax=ax,marker=".",grid=True,c='b',label="band LSST-U") 
df_merge.plot.scatter(x="amxpwv", y="d_adug_mmag",ax=ax,marker=".",grid=True,c='g',label="band LSST-G") 
df_merge.plot.scatter(x="amxpwv", y="d_adur_mmag",ax=ax,marker=".",grid=True,c='r',label="band LSST-R") 
df_merge.plot.scatter(x="amxpwv", y="d_adui_mmag",ax=ax,marker=".",grid=True,c='orange',label="band LSST-I") 
df_merge.plot.scatter(x="amxpwv", y="d_aduz_mmag",ax=ax,marker=".",grid=True,c='k',label="band LSST-Z") 
df_merge.plot.scatter(x="amxpwv", y="d_aduy_mmag",ax=ax,marker=".",grid=True,c='grey',label="band LSST-Y") 

ax.set_xlabel("$airmass \\times PWV (mm)$")
ax.set_ylabel("rel. extinction (mmag)")
ax.set_title(f"relative extinction for a flat SED wrt standard atmosphere at airmass = {am0}")
plt.show()

### Relative Extinction in -2.5 log(ADU) relative to standard atmosphere at observed airmass vs (PWV)

- Relative Extinction means subtract the standard atmosphere ADU at observed airmass

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6),layout="constrained")

df_merge.plot.scatter(x="pwv", y="d_aduu_mmag",ax=ax,marker=".",grid=True,c='b',label="band LSST-U") 
df_merge.plot.scatter(x="pwv", y="d_adug_mmag",ax=ax,marker=".",grid=True,c='g',label="band LSST-G") 
df_merge.plot.scatter(x="pwv", y="d_adur_mmag",ax=ax,marker=".",grid=True,c='r',label="band LSST-R") 
df_merge.plot.scatter(x="pwv", y="d_adui_mmag",ax=ax,marker=".",grid=True,c='orange',label="band LSST-I") 
df_merge.plot.scatter(x="pwv", y="d_aduz_mmag",ax=ax,marker=".",grid=True,c='k',label="band LSST-Z") 
df_merge.plot.scatter(x="pwv", y="d_aduy_mmag",ax=ax,marker=".",grid=True,c='grey',label="band LSST-Y") 

ax.set_xlabel("$PWV (mm)$")
ax.set_ylabel("rel. extinction (mmag)")
ax.set_title(f"relative extinction for a flat SED wrt standard atmosphere at airmass = {am0}")

plt.show()

### Relative Extinction in -2.5 log(ADU) relative to standard atmosphere at observed airmass vs airmass

- Relative Extinction means subtract the standard atmosphere ADU at observed airmass

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6),layout="constrained")

df_merge.plot.scatter(x="am", y="d_aduu_mmag",ax=ax,marker="^",grid=True,c='b',label="band LSST-U") 
df_merge.plot.scatter(x="am", y="d_adug_mmag",ax=ax,marker="v",grid=True,c='g',label="band LSST-G") 
df_merge.plot.scatter(x="am", y="d_adur_mmag",ax=ax,marker="s",s=10,grid=True,c='r',label="band LSST-R") 
df_merge.plot.scatter(x="am", y="d_adui_mmag",ax=ax,marker="o",s=50,grid=True,c='orange',label="band LSST-I") 
df_merge.plot.scatter(x="am", y="d_aduz_mmag",ax=ax,marker="+",s=50,grid=True,c='k',label="band LSST-Z") 
df_merge.plot.scatter(x="am", y="d_aduy_mmag",ax=ax,marker=".",s=50,grid=True,c='grey',label="band LSST-Y") 

ax.set_xlabel("$airmass$")
ax.set_ylabel("rel. extinction (mmag)")
ax.set_title(f"relative extinction for a flat SED wrt standard atmosphere at airmass = {am0}")

plt.show()

### Extinction in -2.5 log(ADU) relative to standard atmosphere at observed airmass vs (airmass x PWV)

- DO not subtract the standard atmosphere ADU at observed airmass

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6),layout="constrained")

df_merge.plot.scatter(x="am", y="aduu_std",ax=ax,marker=".",grid=True,c='b',label="band U") 
df_merge.plot.scatter(x="am", y="adug_std",ax=ax,marker=".",grid=True,c='g',label="band LSST-G") 
df_merge.plot.scatter(x="am", y="adur_std",ax=ax,marker=".",grid=True,c='r',label="band LSST-R") 

df_merge.plot.scatter(x="am", y="adui_std",ax=ax,marker=".",grid=True,c='orange',label="band LSST-I") 
df_merge.plot.scatter(x="am", y="aduz_std",ax=ax,marker=".",grid=True,c='k',label="band LSST-Z") 
df_merge.plot.scatter(x="am", y="aduy_std",ax=ax,marker=".",grid=True,c='grey',label="band LSST-Y") 

ax.set_xlabel("$airmass$")
ax.set_ylabel("Extinction (mag)")
ax.set_title(f"Extinction for a flat SED wrt standard atmosphere at airmass = {am0}")

plt.show()

## Remove Rayleigh at the observed airmass

- Remove the instrumental magnitudes at observed airmass and PWV = 0 mm

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6),layout="constrained")

df_merge.plot.scatter(x="amxpwv", y="d_adu_0u_mmag",ax=ax,marker=".",grid=True,c='b',label="band LSST-U") 
df_merge.plot.scatter(x="amxpwv", y="d_adu_0g_mmag",ax=ax,marker=".",grid=True,c='g',label="band LSST-G") 
df_merge.plot.scatter(x="amxpwv", y="d_adu_0r_mmag",ax=ax,marker=".",grid=True,c='r',label="band LSST-R") 
df_merge.plot.scatter(x="amxpwv", y="d_adu_0i_mmag",ax=ax,marker=".",grid=True,c='orange',label="band LSST-I") 
df_merge.plot.scatter(x="amxpwv", y="d_adu_0z_mmag",ax=ax,marker=".",grid=True,c='k',label="band LSST-Z") 
df_merge.plot.scatter(x="amxpwv", y="d_adu_0y_mmag",ax=ax,marker=".",grid=True,c='grey',label="band LSST-Y") 

ax.set_xlabel("$airmass \\times PWV (mm)$")
ax.set_ylabel("rel. extinction (mmag)")
ax.set_title(f"relative extinction for a flat SED wrt pwv = 0 mm")

plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6),layout="constrained")

#df_merge.plot.scatter(x="am", y="d_adu_0u",ax=ax,marker=".",grid=True,c='b',label="band LSST-U") 
#df_merge.plot.scatter(x="am", y="d_adu_0g",ax=ax,marker=".",grid=True,c='g',label="band LSST-G") 
df_merge.plot.scatter(x="amxpwv", y="d_adu_0r_mmag",ax=ax,marker=".",grid=True,c='r',label="band LSST-R") 
df_merge.plot.scatter(x="amxpwv", y="d_adu_0i_mmag",ax=ax,marker=".",grid=True,c='orange',label="band LSST-I") 
df_merge.plot.scatter(x="amxpwv", y="d_adu_0z_mmag",ax=ax,marker=".",grid=True,c='k',label="band LSST-Z") 
#df_merge.plot.scatter(x="am", y="d_adu_0y",ax=ax,marker=".",grid=True,c='grey',label="band LSST-Y") 

ax.set_xlabel("$airmass \\times pwv$")
ax.set_ylabel("rel. extinction (mmag)")
ax.set_title(f"relative extinction for a flat SED wrt pwv = 0 mm")

plt.show()

## Save file

In [ ]:
df_merge.to_csv("extinctioninbands_o2xairmass.csv")